# Representacion no estructurada de canciones de los 80s

## Importacion de datos

In [ ]:
%pip install scikit-learn matplotlib pandas seaborn nltk pyLDAvis spacy

In [29]:
# Importacion de las letras desde el directorio de letras
import os
import pandas as pd

# Crear dataframe con las letras: Cancion, Letra y Artista
canciones = []
for file in os.listdir('letras'):
    with open(f'letras/{file}', encoding='utf-8') as f:
        cancion = ' '.join(file.split('.')[0:-1]).strip()
        nombre = ' '.join(cancion.split('-')[0:-1]).strip()
        artista = cancion.split('-')[-1].strip()
        letras = f.read()
        canciones.append([nombre, artista, letras])

canciones = pd.DataFrame(canciones, columns=['Cancion', 'Artista', 'Letra'])

canciones.to_csv('canciones.csv', index=False)

canciones.head()

,Cancion,Artista,Letra
0,A esa,Pimpinela,"L VEN AQUI, QUIERO DECIRTE ALGO...\n\nA esa, q..."
1,A quién le importa,Alaska y Dinarama,La gente me señala\n\nme apuntan con el dedo\n...
2,Amante Bandido,Miguel Bosé,Yo seré el viento que va\n\nnavegare por tu os...
3,Amor Eterno,Rocío Dúrcal,Tu eres la tristeza de mis ojos \nque lloran e...
4,Ausencia,Héctor Lavoe,Ha terminado otro capítulo en mi vida \nLa muj...


## Representacion TF-IDF

### Entendimiento y preprocesamiento de datos

In [30]:
letras = canciones['Letra'].values.tolist()
len(letras)

105

Importacion de la libreria de stopwords y definicion de una funcion para filtrar las canciones.

In [31]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
misstop=stopwords.words('spanish')+["á","Y"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
def filtrado(texto):
    filtrados=[word for word in texto if word not in misstop]
    return(filtrados)

La limpieza incluira:

**a)** pasar a minúsculas

**b)** retirar páginas web

**c)** quitar saltos de línea y tabuladores

**d)** retirar stopwords

**e)** retirar menciones

**f)** unir todo de nuevo

**g)** eliminar letras solas


In [33]:
import re
nltk.download("punkt")
nltk.download('punkt_tab')
filtradito=[]
# Eliminar menciones y páginas web
for i in range(len(letras)):
    letras[i] = re.sub("\""," ",letras[i])
    letras[i] = re.sub ("\n|\t"," ",letras[i])
    letras[i]=letras[i].lower()
    letras[i] = re.sub(r'\s+[a-zA-Z]\s+', ' ', letras[i])
    letras[i] = re.sub(r'\xad+', ' ', letras[i], flags=re.I)
    breve=nltk.tokenize.word_tokenize(letras[i],language="spanish")
    tempfilt=filtrado(breve)
    filtradito.append(tempfilt)
    
unidito=[]
for element in filtradito:
    unidito.append(" ".join(element))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
unidito[0]

'l ven aqui , quiero decirte ... , aparta , roba tiempo , alma cuerpo , ve dile ... quieres ? venga , valor , muestre cara hable frente si quiere amor ... ? , contigo va vestida princesa , , hace preguntas siempre está¡ dispuesta , , vete dile ... ? venga ... ? doy lugar ... quieres probar ? recoja mesa , lave ropa todas miserias ... quieres demostrar ? venga , juegue ... vas conseguir ? quiero ver si capaz darte cosas di ... , ... vete dile , venga ... , pone tan mal capaz hacerme volver vivir ilusiones perdidas . , hace hablar debo cosas hace tiempo das ... , puede costar hacerte feliz hora día ? , toca vivir ninguna tristeza , alegrí a. vete dile .. ? venga ... ? doy lugar ... quieres probar ? recoja mesa , lave ropa todas miserias ... quieres demostrar ? venga , juegue ... vas conseguir ? quiero ver si capaz darte cosas di ... venga ... ... doy lugar ... quieres probar ... recoja mesa lave ropa todas miserias quieres demostrar ... venga juegue ... vas conseguir ... quiero ver si ca

### Determinacion de Topicos

Importacion de librerias para topicos de sklearn

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

Estraccion de topicos

In [36]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [37]:
vectores=TfidfVectorizer(max_df=0.9,min_df=2,max_features=1000)
tfslimpio=vectores.fit_transform(unidito)
nombreslimpio=vectores.get_feature_names_out()
nmflimpio = NMF(n_components=5, random_state=1, l1_ratio=.5, init='nndsvd').fit(tfslimpio)
no_top_words=10
display_topics(nmflimpio, nombreslimpio, no_top_words)

Topic 0:
si quiero voy amor sé vos vez noche siempre nadie
Topic 1:
vida pido why toda asi así amor ahora prefiero nunca
Topic 2:
oh bajo parte playa calienta cerca aquí luna sol busco
Topic 3:
desaparecer ayer amor muralla parado divide mirando vuelve viene realidad
Topic 4:
sacarla ayudarla puedo aca pozo escucho abajo mas ciego mirarla


In [14]:
len(nombreslimpio)

790

Utilizacion de conteos para LDA (Latent Dirilchet Allocation)

In [15]:
paralda = CountVectorizer(max_df=0.9, min_df=2, max_features=1000)
tflda=paralda.fit_transform(unidito)
nombreslda=paralda.get_feature_names_out()
lda = LatentDirichletAllocation(n_components=8, max_iter=3, learning_method='online', learning_offset=50.,random_state=0).fit(tflda)
id_topico=lda.fit_transform(tflda)
display_topics(lda,nombreslda,no_top_words)

Topic 0:
vida mundo ay solo toda tan así oh si llorar
Topic 1:
amor eu debajo um rock não ayer desaparecer corazón dia
Topic 2:
oh nadie vete bis puede verás jamás hombre luna lleva
Topic 3:
na quiero amor noches vida cinco pido luna noche vez
Topic 4:
amo je lluvia si nadie fríos besos días gota non
Topic 5:
importa asi si pasa siempre nunca amor ahora manos primavera
Topic 6:
si amor sé ahora quiero puedo nunca siempre vida gente
Topic 7:
si ven mas quiero voy nene van solo amor ser


Pertenencia de cada documento a su topico

In [16]:
doc_topic = nmflimpio.transform(tfslimpio)
topic_most_pr=[]
for n in range(doc_topic.shape[0]):
    topic_most_pr.append(doc_topic[n].argmax())

In [17]:
canciones["nmf"]=topic_most_pr
ver_cerveza=canciones[["Cancion","nmf"]]
display(ver_cerveza)

,Cancion,nmf
0,A esa,0
1,A quién le importa,1
2,Amante Bandido,0
3,Amor Eterno,0
4,Ausencia,0
...,...,...
100,Y cómo es él,1
101,Yo No Te Pido La Luna,1
102,Yo te avisé,0
103,Ámame en cámara lenta,1


### Visualizacion de resultados

Gensim y pyLDAvis para ver los resultados de LDA

In [ ]:
%pip install --upgrade numpy==2.0.0
%pip install --upgrade scipy==1.9.3
%pip install --upgrade gensim

In [19]:
import gensim
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [20]:
import spacy
spacy.cli.download("es_core_news_sm")
nlp=spacy.load("es_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [21]:
preparado = []
for texto in letras:
  doc = nlp(texto) 
  preparado.append([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV'] and token.text!="cerveza" ])
preparado[0:2]

[['ver',
  'aqui',
  'querer',
  'decir tú',
  'apartar',
  'robar',
  'tiempo',
  'alma',
  'cuerpo',
  'ver',
  'dile',
  'quier',
  'venir',
  'tener',
  'valor',
  'mostrar',
  'cara',
  'hablar',
  'frente',
  'querer',
  'amor',
  'estar',
  'vestido',
  'princesa',
  'no',
  'hacer',
  'pregunta',
  'siempre',
  'dispuesto',
  'vete',
  'dile',
  'venir',
  'dar',
  'lugar',
  'querer',
  'probar',
  'recojar',
  'mesa',
  'lave',
  'ropar',
  'miseria',
  'querer',
  'demostrar',
  'venir',
  'jugar',
  'ir',
  'conseguir',
  'querer',
  'ver',
  'capaz',
  'darte',
  'cosa',
  'dar',
  'vete',
  'dile',
  'venir',
  'poner',
  'tanto',
  'mal',
  'capaz',
  'hacer yo',
  'volver',
  'vivir',
  'ilusión',
  'perdido',
  'hacer',
  'hablar',
  'deber',
  'cosa',
  'hacer',
  'tiempo',
  'ya',
  'no',
  'dar',
  'costar',
  'hacerte',
  'feliz',
  'hora',
  'día',
  'no',
  'tocar',
  'vivir',
  'tristeza',
  'alegrí',
  'a.',
  'vete',
  'dile',
  'venir',
  'dar',
  'lugar',
  

Preparacion del conteo y definicion de un diccionario a gensim

In [22]:
id2word=Dictionary(preparado)
corpus = [id2word.doc2bow(element) for element in preparado]

Tecnica basada en PMI que calcula la coherencia del numero de topicos.

In [23]:
coherencias=[]
for topicos in range(1,10):
  modelito=gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=topicos,random_state=100,update_every=1,chunksize=40,passes=10,alpha='auto',per_word_topics=True)
  coherence_model_lda = CoherenceModel(model=modelito, texts=preparado, dictionary=id2word, coherence='c_v')
  coherencias.append(coherence_model_lda.get_coherence())
print(coherencias)

c:\Users\esteb\Documents\Trabajos U\FAM\Music-80s-representation\.venv\Lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
c:\Users\esteb\Documents\Trabajos U\FAM\Music-80s-representation\.venv\Lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
c:\Users\esteb\Documents\Trabajos U\FAM\Music-80s-representation\.venv\Lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https:/

[0.2557710542554721, 0.29580164197489833, 0.4040402440152573, 0.35672570090001116, 0.39362951266096946, 0.37092142977304937, 0.37964277764752935, 0.37810276174314733, 0.38052115590623664]


El resultado sugiere tres topicos

In [24]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=40,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
from pprint import pprint

In [26]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.067*"no" + 0.018*"vida" + 0.014*"tener" + 0.014*"amor" + 0.013*"más" + '
  '0.011*"amar" + 0.010*"saber" + 0.010*"siempre" + 0.009*"nunca" + '
  '0.009*"ahora"'),
 (1,
  '0.036*"no" + 0.025*"querer" + 0.011*"amor" + 0.010*"pedir" + 0.009*"ver" + '
  '0.009*"noche" + 0.007*"más" + 0.007*"bastar" + 0.007*"ya" + 0.007*"ir"'),
 (2,
  '0.017*"no" + 0.011*"ir" + 0.008*"amor" + 0.008*"signo" + 0.007*"querer" + '
  '0.006*"eu" + 0.005*"agüita" + 0.005*"amarillo" + 0.005*"dar" + '
  '0.005*"creer"')]


In [27]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.137756  0.042505       1        1  49.379100
1     -0.027006 -0.126131       2        1  27.178369
2     -0.110750  0.083626       3        1  23.442531, topic_info=        Term        Freq       Total Category  logprob  loglift
43    querer  113.000000  113.000000  Default  30.0000  30.0000
104     vida   89.000000   89.000000  Default  29.0000  29.0000
2069   signo   17.000000   17.000000  Default  28.0000  28.0000
206    pedir   43.000000   43.000000  Default  27.0000  27.0000
282   bastar   18.000000   18.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
49     tanto    8.023775   47.629459   Topic3  -5.6002  -0.3304
213    saber    8.331550   60.567316   Topic3  -5.5626  -0.5331
149      hoy    6.718599   23.396564   Topic3  -5.7777   0.2029
240     aquí    6.380291   14.675566   Topic3  -5.8294   0.6176
24     hacer    6.212545   61.094474   Topic3  -5.8561  -0.8352

[197 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1583      3  0.964993  agüita
337       2  0.959029     ahh
63        1  0.926874   ahora
63        2  0.045213   ahora
63        3  0.022607   ahora
...     ...       ...     ...
61        2  0.352728      ya
61        3  0.117576      ya
2354      2  0.919636   ámame
2111      1  0.837851       �
2111      2  0.162165       �

[243 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])